In [1]:
import pandas as pd
import os
from sksurgerynditracker.nditracker import NDITracker
import numpy as np

In [2]:
# tool tip offset for clear pen obtained from NDI Toolbox using Pivoting
clear_pen_tool_tip_offset = np.array([-1.62, 2.58, 17.80, 1]) 

# Initialize the NDI tracker
SETTINGS = {
    "tracker type": "aurora",
    "port": "COM3"
}
tracker = NDITracker(SETTINGS)
tracker.start_tracking()

In [3]:
def get_tooltip_data(tool_tip_offset):
    port_handles, timestamps, framenumbers, tracking, quality = tracker.get_frame()
    tool_tip = np.dot(tracking[0], tool_tip_offset)
    x_pos = tool_tip[0]
    y_pos = tool_tip[1]
    z_pos = tool_tip[2]
    return x_pos, y_pos, z_pos

In [4]:
# Load target fiducial's positions
fcsv_file = 'MARKERS.fcsv'
target_fiducials = pd.read_csv("C:\\Users\\f007wsq\\Desktop\\Calibration Protocol - Aurora\\Calibration Protocol" + "\\"  + fcsv_file, comment='#', header=None)

In [ ]:
real_pos = []
i = 0
while i < len(target_fiducials):
    user_input = input("Press Enter to Collect Points：")
        
    if user_input.lower() == 'Exit':
        break

    if user_input == '':  # 如果按回车键
        x, y, z = get_tooltip_data(clear_pen_tool_tip_offset)  # 获取当前点在FG中的位置
        if np.isnan(x) or np.isnan(y) or np.isnan(z):
            print("Collect again")
        else:
            real_pos.append([x, y, z])  # 保存采集的数据       
            i = i + 1
            print(f"Collected Point: {i}")
            print(f"Position: {x, y, z}")
            

In [6]:
from scipy.spatial.transform import Rotation
def compute_rigid_transform(P_target, P_aurora):
    """
    计算从 target 坐标系到 aurora 坐标系的刚体变换 (R, T)
    
    :param P_target: ndarray (N,3)，target 坐标系下 fiducials 的 3D 坐标
    :param P_aurora: ndarray (N,3)，aurora 坐标系下 fiducials 的 3D 坐标
    :return: R (3x3 rotation matrix), T (3x1 translation vector)
    """
    # 计算两个点集的质心
    centroid_target = np.mean(P_target, axis=0)
    centroid_aurora = np.mean(P_aurora, axis=0)

    # 去中心化（中心化点集）
    P_target_centered = P_target - centroid_target
    P_aurora_centered = P_aurora - centroid_aurora

    # 计算协方差矩阵 H
    H = P_target_centered.T @ P_aurora_centered

    # 进行 SVD 分解
    U, _, Vt = np.linalg.svd(H)
    
    # 计算旋转矩阵 R
    R = Vt.T @ U.T

    # 处理可能的反射问题（保证 R 是正交矩阵，det(R) = 1）
    if np.linalg.det(R) < 0:
        Vt[-1, :] *= -1
        R = Vt.T @ U.T

    # 计算平移向量 T
    T = centroid_aurora - R @ centroid_target

    return R, T

In [ ]:
P_target = []
target_fiducials = np.array(target_fiducials)
for i in range(len(target_fiducials)):
    P_target.append(target_fiducials[i][1:4])
P_target = np.array(P_target, dtype=np.float64)
print(P_target)

In [ ]:
P_aurora = np.array(real_pos, dtype=np.float64)
print(P_aurora)

In [9]:
R, T = compute_rigid_transform(P_target, P_aurora)


In [10]:
R, T = compute_rigid_transform(P_target, P_aurora)


In [11]:
def create_transformation_matrix(R, T):
    """
    将旋转矩阵 R (3x3) 和平移向量 T (3x1) 组合成 4x4 变换矩阵
    """
    transformation_matrix = np.eye(4)  # 创建一个 4x4 单位矩阵
    transformation_matrix[:3, :3] = R  # 设置旋转部分
    transformation_matrix[:3, 3] = T.flatten()  # 设置平移部分
    return transformation_matrix

In [ ]:
transformation_matrix = create_transformation_matrix(R, T)
print(transformation_matrix)